In [ ]:
%matplotlib inline
from datetime import date, datetime, timedelta, time
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

In [ ]:
from simlightcurve.curves import GaussExp as Gred
from simlightcurve.solvers import find_rise_t, find_peak
from scipy.stats import norm
import scipy.optimize as op
import emcee

In [ ]:
seaborn.set_context('poster')
current_palette = seaborn.color_palette()

In [ ]:
sigma=0.1
detection_thresh = 5.
analysis_thresh = 3.

$\qquad =  - 0.5 \sum_{i=1}^N \left[ ln[2\pi\sigma^2] + [x_i - \alpha]^2 / \sigma^2  \right]$

In [ ]:
amp_mean = 1.15
amp_sigma = 0.15
amplitude = norm(loc=amp_mean, scale=amp_sigma)


def gaussian_logpdf(x, mean, sigma):
    return -0.5*(np.log(2 * np.pi * sigma ** 2) + ((x - mean)/sigma) ** 2 )

amplitude_tail_cut = 1e-5
amp_minmax = (amplitude.ppf(amplitude_tail_cut), amplitude.ppf(1-amplitude_tail_cut))

def amp_prior(x):    
    if amp_minmax[0]<=x<=amp_minmax[1]:
        return gaussian_logpdf(x, amp_mean, amp_sigma)
    else:
        return -np.inf
    

In [ ]:
np_rvs = norm(loc=amp_mean, scale=amp_sigma)
print gaussian_logpdf(0.6, amp_mean,amp_sigma)
print np_rvs.logpdf(0.6)


In [ ]:
inputs = np.linspace(amp_minmax[0], amp_minmax[1], 1e6)

In [ ]:
%timeit gaussian_logpdf(inputs, amp_mean,amp_sigma)

In [ ]:
%timeit np_rvs.logpdf(inputs)

In [ ]:
amp_minmax

In [ ]:
def t0_prior(t0):
#     return 0.0
    if -60<= t0 < 40.:
        return 0.0
    return -np.inf

In [ ]:
true_t0 = 5 #+np.random.random()*5
true_amp = norm(loc=amp_mean, scale=amp_sigma).rvs()
print true_t0,true_amp
true_curve = Gred(amplitude=true_amp, rise_tau=5,decay_tau=15, t0=true_t0)
prior_curve = Gred(amplitude=amp_mean, rise_tau=5,decay_tau=15, t0=true_t0)

In [ ]:
tstep = 1.5
sim_epochs = np.arange(start = -30., stop=50, step=tstep)
sim_data = true_curve(sim_epochs)+norm(loc=0, scale=sigma).rvs(size=len(sim_epochs))

sim_data = pd.Series(data=sim_data, index=sim_epochs)
# sim_data

In [ ]:
threshold = sigma*detection_thresh

# detection_time = find_rise_t(true_curve, threshold=threshold, 
#                             t_min=true_curve.t0-2*true_curve.rise_tau, t_max=find_peak(true_curve,true_curve.t0)[0]) 
print "Thresh:", threshold
detectable = sim_data[sim_data>sigma*detection_thresh]
monitorable = sim_data[sim_data>sigma*analysis_thresh]
first_detection_epoch = detectable.index[0]
usable_data = monitorable[first_detection_epoch:]
# usable_data

In [ ]:
detectable.index[0]

In [ ]:
seaborn.palplot(current_palette)

In [ ]:
tsteps= np.linspace(-30,50, 1000)
plt.plot(tsteps, true_curve(tsteps),c='g', ls='--',label='True')
plt.plot(tsteps, prior_curve(tsteps),c='b', ls=':', label='Priori')
# plt.plot(tsteps, true_curve(tsteps)+sigma, ls=':',c='g')
# plt.plot(tsteps, true_curve(tsteps)-sigma, ls=':',c='g')

plt.scatter(sim_data.index, sim_data,c=current_palette[2])

plt.axhline(sigma*detection_thresh, ls='--', label='Detection')
plt.axhline(sigma*analysis_thresh, ls='-.', label='Analysis')

plt.scatter(usable_data.index, usable_data,c=current_palette[1],s=55)

# plt.yscale('log')
# plt.axhline(true_amp, ls=':')

# plt.axvline(true_t0, ls='--')
plt.ylim(-0.2,1.6)
plt.legend()


In [ ]:
# support = np.linspace(amplitude.ppf(0.01), amplitude.ppf(.99), 100)
# plt.plot(support, amplitude.pdf(support))
# # plt.plot(support, np.log(amplitude.pdf(support)))
# plt.plot(support, amp_prior(support))

In [ ]:
# class CurveEnsemble(object):
#     def __init__(self, curve, logpriors):
#         self.curve

In [ ]:
#Detect, class2:
n_data_epochs = 1
obs_data = usable_data.iloc[:n_data_epochs]
obs_data

In [ ]:
# comparison = pd.DataFrame(index=usable_data.index, data={'noisy':usable_data, 'true':true_curve(usable_data.index)})
# comparison['err'] = comparison.true - comparison.noisy
# comparison.plot()

In [ ]:
fit_curve_pars=dict(zip(prior_curve.param_names,prior_curve.parameters))
fit_curve_pars['t0']=0.0 
# c2_pars['t']=0.0
# class2.eval(**c2_pars)
zero_t0_curve = prior_curve.__class__(**fit_curve_pars)

In [ ]:
detection_t0_offset = find_rise_t(zero_t0_curve, threshold=obs_data.iloc[0], 
                                     t_min=-2*prior_curve.rise_tau, t_max=0)
t0_guess = obs_data.index[0]-detection_t0_offset
t0_guess

In [ ]:
def log_likelihood(theta, obs_data, model, theta_params, fixed_params, obs_sigma):
    model_pars = dict(zip(theta_params,theta))
    model_pars.update(fixed_params)
    intrinsic_fluxes = model.evaluate(obs_data.index, **model_pars)
#     return -0.5 * np.sum(np.log(2 * np.pi * sigma ** 2) + ((noisy_fluxes-intrinsic_fluxes) /sigma) ** 2)
    return -0.5 * np.sum( ((obs_data.values-intrinsic_fluxes) /obs_sigma) ** 2)

In [ ]:
prior_curve.param_names

In [ ]:
theta_params = ('t0',)
fixed_params = dict(zip(prior_curve.param_names,prior_curve.parameters))
for par in theta_params: 
    fixed_params.pop(par)
fixed_params

In [ ]:
neg_likelihood = lambda *args: -log_likelihood(*args)
results = op.minimize(neg_likelihood, -10, args=(obs_data, prior_curve, theta_params, fixed_params, sigma))
print true_t0
print results.x, results.x - true_t0
print t0_guess, results.x, t0_guess - results.x

In [ ]:
plt.plot(tsteps,true_curve(tsteps), ls='--', label='true')

fit_curve_pars=dict(zip(prior_curve.param_names,prior_curve.parameters))
fit_curve_pars['t0']=t0_guess

plt.plot(tsteps,prior_curve.__class__(**fit_curve_pars)(tsteps), ls='--', label='rise_time_guess')
fit_curve_pars['t0']=results.x
plt.plot(tsteps,prior_curve.__class__(**fit_curve_pars)(tsteps), ls='--', label='Init MAP_fit')
# plt.plot(tsteps,true_class2.__class__(**c2_pars)(tsteps), ls='--', label='class2_model_fit_decline')
plt.scatter(obs_data.index, obs_data, s=50)
plt.errorbar(obs_data.index, obs_data, yerr=1*sigma, c=current_palette[2], linewidth=0,elinewidth=3, ms=16, marker='.')
plt.errorbar(obs_data.index, obs_data, yerr=2*sigma, c=current_palette[2], linewidth=0,elinewidth=1.5)
plt.ylim(0,1.6)
plt.legend()

In [ ]:
prior_curve.param_names

In [ ]:
theta_params = ('t0', 'amplitude')
theta_priors = (t0_prior, amp_prior)
fixed_params = dict(zip(prior_curve.param_names,prior_curve.parameters))
for par in theta_params: 
    fixed_params.pop(par)
print theta_params
print fixed_params

In [ ]:
def log_prior(theta):
    prior = 0.0
    for f,val in zip(theta_priors,theta):
        prior+=f(val)
    return prior

def log_prob(theta, obs_data, model, theta_params, fixed_params, obs_sigma):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        prob = -np.inf
    else:
        prob = log_likelihood(theta, obs_data, model, theta_params, fixed_params, obs_sigma)
    return prob

In [ ]:
theta_init = np.asarray((t0_guess, prior_curve.amplitude.value))
theta_init

In [ ]:
ndim = len(theta_params)  # number of parameters in the model
nwalkers = 50  # number of MCMC walkers
nsteps = 500  # number of MCMC steps to take
# nsteps = 2  # number of MCMC steps to take
# nburn=1
nthreads=4
#for PT
ntemps=20


# starting_point = results.x

# pos = [starting_point+ 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
# pos = [starting_point+ 1e-4*np.random.randn(ndim) for i in range(nwalkers)]

prop_scale=4

nwalkers= nwalkers*5
en_sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, 
                                a=prop_scale, 
                                args=(obs_data, prior_curve, theta_params, fixed_params, sigma),
                                threads=nthreads)


en_theta_init_ball = theta_init+ 1e-4*np.random.randn(nwalkers*ndim).reshape(nwalkers,ndim)

#PT needs extra Ntemps dimension:

# pt_theta_init_ball = np.random.uniform(low=-1.0, high=1.0, size=(ntemps, nwalkers, ndim))
pt_t0_init = theta_init+ 1e-4*np.random.randn(ndim*nwalkers*ntemps).reshape(ntemps,nwalkers,ndim)
pt_sampler = emcee.PTSampler(ntemps,
                          nwalkers, ndim, 
                          logl=log_likelihood,
                          logp=log_prior, 
                          a=prop_scale, 
                          loglargs=(obs_data, prior_curve, theta_params, fixed_params, sigma),
                          logpargs=(),
                          threads=nthreads
                          )
sampler=en_sampler
theta_init_ball = en_theta_init_ball

# sampler=pt_sampler
# theta_init_ball = pt_theta_init_ball

In [ ]:
# theta_init_ball

In [ ]:
sampler.reset()
_=sampler.run_mcmc(theta_init_ball, nsteps)
print "Done"

In [ ]:
nburn=250
acorr = np.ceil(np.max(sampler.get_autocorr_time()))
print "Acorr:", acorr
#Ensemble
samples=sampler.chain[:,nburn::acorr,:]
samples.reshape(-1,ndim)
plotchain=sampler.chain
acceptance = np.median(sampler.acceptance_fraction)

#PT
# plotchain=sampler.chain[0]
# samples=sampler.chain[0][:,nburn::acorr,:].ravel()
# acceptance = np.median(sampler.acceptance_fraction[0])

print "Acceptance:", acceptance


In [ ]:
varindex = 1
plt.subplot(2,1,1)
for walker in plotchain[:,:,varindex]:
    plt.plot(walker)
plt.axvline(nburn, ls=':', color='k')
print "Acorr",sampler.get_autocorr_time()
print "Acceptance",acceptance
plt.subplot(2,1,2)
for walker in plotchain[:,::acorr,varindex]:
    plt.plot(walker)

In [ ]:
sampler.acor

In [ ]:
varindex=0
plt.hist(plotchain[:,nburn::1,varindex].ravel(),normed=True,alpha=0.8)
plt.hist(plotchain[:,nburn::acorr,varindex].ravel(),normed=True,alpha=0.5)

In [ ]:
samples = samples.ravel().reshape(-1,ndim)
samples.shape

In [ ]:
plt.hexbin(samples[:,0], samples[:,1])

In [ ]:
import triangle

In [ ]:
figure = triangle.corner(samples, truths=[true_curve.t0.value, true_curve.amplitude.value])

In [ ]:
print theta_params
print fixed_params

In [ ]:
print len(samples)
subsamples = samples[np.random.randint(len(samples),size=100)]

    
forecasts = []

for theta in subsamples:
    model_pars = dict(zip(theta_params,theta))
    model_pars.update(fixed_params)
    values = prior_curve.evaluate(tsteps, **model_pars)
    forecasts.append(values)

forecasts = np.asarray(forecasts)
forecasts.shape

In [ ]:
# np.max(subsamples_t0[:,1])
# subsamples

In [ ]:
#Forecast:
# t_forecast= 15
# forecast_data=[zero_t0_class2(t_forecast - t0) for t0 in samples]

In [ ]:
print np.max(subsamples[:,1])
print np.max(forecasts)

In [ ]:
from matplotlib import gridspec

fig = plt.figure() 
# fig, axes = plt.subplots(nrows=1, ncols=2,
# #                          sharey=True, 
# #                          figsize=(18,6)
#                          )

seaborn.set_context('poster')
gs = gridspec.GridSpec(1,2, width_ratios=[3,1])

ts_axes = plt.subplot(gs[0])

seaborn.tsplot( forecasts,  tsteps, err_style="unit_traces", ax=ts_axes)
# seaborn.tsplot(curves,  tsteps, 
#                err_style="ci_band",
#                color='k', ax=ts_axes)
ts_axes.plot(tsteps,true_curve(tsteps), ls='--', c='y',label='true', lw='5')

ts_axes.scatter(obs_data.index,obs_data, c='r', s=200,zorder=10)
# ts_axes.errorbar(t_data,noisy_flux, c='y',yerr=sigma,zorder=10)
# ts_axes.axvline(t_forecast, ls=':')
# ts_axes.axhline(np.mean(forecast_data), ls=':')

# hist_axes = plt.subplot(gs[1])
# hist_axes.hist(forecast_data, orientation='horizontal')
# _=hist_axes.set_ylim(ts_axes.get_ylim())
plt.legend()
# plt.savefig('test.png')